In [ ]:
import xarray as xr
import geopandas as gpd
import glob
import datetime
import pandas as pd
import panel as pn
import numpy as np
import param as pm
import holoviews as hv
import geoviews as gv
import geoviews.tile_sources as gts
from holoviews.element import tiles as hvts
from holoviews.operation.datashader import rasterize, shade, spread, aggregate
import datashader as ds
from collections import OrderedDict as odict
from holoviews import opts
import matplotlib.pyplot as plt
import intake
from datashader.utils import lnglat_to_meters
from distributed import Client, LocalCluster
renderer = hv.renderer('bokeh')
hv.extension('bokeh', logo=False)
pn.extension(logo=False)
import hvplot.dask
from cartopy import crs as ccrs

Start a Local Dask Cluster

In [ ]:
cluster = LocalCluster()
cl = Client(cluster)
cl

Some Data Preprocessing Functions

In [ ]:
def coord2num(c,negify=False):
    if negify:
        return(float(c[0:-1])*-1.)
    else:
        return(float(c[0:-1]))
def smithtech_datetime(df):
    df['datetime'] = pd.to_datetime(df.DATE*1000000.+df.TIME,format='%y%m%d%H%M%S',errors='coerce')
    return(df)

def lotek_datetime(df):
    df['datetime'] = pd.to_datetime(df['GMT Time'],format="%m/%d/%Y %H:%M:%S")
    return(df)

In [ ]:
reformat_data=False
if reformat_data:
    def coord2num(c):
            return(float(c[0:-1]))

    df = pd.read_csv('/mnt/T/2-Projects/CattleCollars/Data_SmithTech/HPGRS/BRTE_D2/BRTE_FixData_D2.CSV')
    df = df.rename(columns={'LATITUDE N/S':'y','LONGITUDE E/W':'x'})
    df['x'] = df['x'].apply(coord2num)
    df['y'] = df['y'].apply(coord2num)
    df['x'] =df.x*-1.
    df.loc[:, 'x_merc'], df.loc[:, 'y_merc'] = lnglat_to_meters(df.x,df.y)
    df.to_csv('/mnt/T/2-Projects/CattleCollars/Data_SmithTech/HPGRS/BRTE_D2/BRTE_FixData_D2_____V4.csv')
    df.head()

Build the Dashboard

In [ ]:
#intake catalog
catalog = intake.open_catalog('/mnt/c/Users/rowan.gaffney/Projects/CPER_Data_Ecosystem/RRSRU_Cattle_Collar_Catalog_v2_example.yml')

#Define some functions to apply to the data when loaded
kwarg_readcsv = {'GPS_CPER_SmithTech':{'converters':{'LATITUDE N/S':lambda x: coord2num(x,negify=False),
                                                     'LONGITUDE E/W':lambda x: coord2num(x,negify=True)}},
                 'GPS_CPER_SmithTech_LongevityTesting':{'converters':{'LATITUDE N/S':lambda x: coord2num(x,negify=False),
                                                                      'LONGITUDE E/W':lambda x: coord2num(x,negify=True)}}}#,
                 #'Cattle_GPS_HPGRS_SmithTech':{'converters':{'x':lambda x: convert_to_bounds(x,coord_type='lon'),'y':lambda x: convert_to_bounds(x,coord_type='lat')}}}

#Define some of the dashboard options
dsets  = odict([(d[0],catalog[d[0]](csv_kwargs=kwarg_readcsv[d[0]])) if d[0] in kwarg_readcsv.keys() else (d[0],catalog[d[0]]) for d in catalog.items()])
plots  = odict([(catalog[list(dsets.keys())[0]].metadata['plots'][p].get('label',p),p) for p in catalog[list(dsets.keys())[0]].plots])
norms  = odict(Linear='linear', Log='log', Cube_root='cbrt')
cmaps  = odict([(n,plt.get_cmap(n)) for n in ['viridis','viridis_r','plasma','plasma_r','inferno','inferno_r','magma','magma_r','cividis','cividis_r']])
maps   = ['EsriImagery', 'EsriUSATopo', 'EsriTerrain', 'CartoMidnight', 'StamenWatercolor', 'StamenTonerBackground']
bases  = odict([(name, getattr(hvts, name)().relabel(name)) for name in maps])
gopts  = hv.opts.Tiles(responsive=True, bgcolor='black', show_grid=False)

#Dashboard
class Explorer(pm.Parameterized):
    dset          = pm.Selector(dsets,label='Data Set')
    cmap          = pm.Selector(cmaps,label='Color Map')
    basemap       = pm.Selector(bases,label='Base Map')
    data_opacity  = pm.Magnitude(1.0,label='Data Opacity')
    map_opacity   = pm.Magnitude(1.0,label='Map Opacity')
    init_mintime  = datetime.datetime.strptime('20190301','%Y%m%d')
    init_maxtime  = datetime.datetime.strptime('20191001','%Y%m%d')
    d_range       = pm.DateRange(default=(init_mintime,init_maxtime),
                                 bounds=(init_mintime,init_maxtime),
                                 label='Date Range')
    num_obs       = pm.Integer(0,label='Number of Obs.')
    
    def __init__(self, **params):
        super(Explorer, self).__init__(**params)
        self.data=self.load_data()
    
    @pm.depends('dset',watch=True)
    def load_data(self):
        if self.dset.metadata['instrument_type']=='Smith_Tech':
            df = self.dset.to_dask()
            dtype_dict = df.dtypes.to_dict()
            dtype_dict['datetime']='datetime64[ns]'#'M8[ns]'
            df = df.map_partitions(smithtech_datetime,meta=dtype_dict)
        if self.dset.metadata['instrument_type']=='Lotek':
            df = self.dset.to_dask()
            dtype_dict = df.dtypes.to_dict()
            dtype_dict['datetime']='datetime64[ns]'
            df = df.map_partitions(lotek_datetime,meta=dtype_dict)
        self.data=df.persist()
        n_bounds = (self.data.datetime.min().compute().to_pydatetime(),
                    self.data.datetime.max().compute().to_pydatetime())
        self.param['d_range'].bounds = n_bounds
        self.param.set_param(d_range=n_bounds)
    
    @pm.depends('load_data','d_range',watch=True)
    def elem(self):
        df = self.data#
        df = df[(df.datetime>=self.d_range[0])&(df.datetime<=self.d_range[1])]
        self.param['num_obs'].default = len(df)
        self.param.set_param(num_obs=len(df))
        return getattr(df, 'hvplot')(x=self.dset.metadata['x'],
                                     y=self.dset.metadata['y'],
                                     kind='points',
                                     geo=True,
                                     crs=ccrs.PlateCarree())

    @pm.depends('map_opacity', 'basemap')
    def tiles(self):
        return self.basemap.opts(gopts).opts(alpha=self.map_opacity)
    
    @pm.depends('dset',watch=True)
    def viewable(self,**kwargs):
        if self.data is None:
            self.load_data()
        rasterized = rasterize(hv.DynamicMap(self.elem),
                               aggregator='count').opts(colorbar=True,
                                                        logz=True,tools=['hover'],
                                                        clipping_colors={'min': 'transparent'}).apply.opts(alpha=self.param.data_opacity,
                                                                                                           cmap=self.param.cmap).redim.range(Count=(1., None),
                                                                                                                                             x=(-106,-103.5),
                                                                                                                                             y=(40.5,41.3))
        return hv.DynamicMap(self.tiles)*rasterized

explorer = Explorer(name='',)

In [ ]:
#Define the Dashboard/Layout
logo = "https://ltar.ars.usda.gov/wp-content/uploads/2018/10/usda_ltar_logo_header_v3.png"
panel1 = pn.GridSpec(sizing_mode='stretch_both')
panel1[0,0]=pn.Row(logo)
panel1[1:3,0] =  pn.Column(pn.Param(explorer.param.dset),
                           pn.Param(explorer.param.data_opacity),
                           pn.Param(explorer.param.cmap),
                           pn.Param(explorer.param.d_range),
                           pn.WidgetBox(explorer.param.num_obs,),
                           pn.Param(explorer.param.basemap),
                           pn.Param(explorer.param.map_opacity))
panel1[0:5,1:5] =  explorer.viewable()

panel2 = pn.GridSpec(sizing_mode='stretch_both')
panel2[0,0]=pn.Row(logo)
panel2[1,0:4] = catalog.gui.panel

panel = pn.Tabs(('Collar Visualization',pn.Pane(panel1)),('Collar Data Details',pn.Pane(panel2)))

Launch the Dashboard

In [ ]:
pn.serve(panel,show=False)